In [ ]:
import glob
import pandas as pd
import numpy as np
from functools import reduce

In [ ]:
color_azul_amazonico="#004254"
color_amarillo_solar="#FBBB21"

In [ ]:
mailing = pd.read_excel('Base 202108.xlsx')
mailing.columns

In [ ]:
# main_df = pd.read_csv('mailing 261020.csv', sep=";", encoding="ANSI")
# main_df.info()
main_df = pd.DataFrame()
main_df['Area']=mailing['Negocio']
main_df['codigo'] = mailing['Cod Empleado']
main_df['Nome'] = mailing['Empleado']
main_df['email']=mailing['Mail']

In [ ]:
files = []
for file in glob.glob('data/*.csv'):
    files.append(file)

In [ ]:
def clean_file(file):
    df = pd.read_csv(file)
    df[file.replace('data\\','')] = 1
    df[file.replace('data\\','')]=df[file.replace('data\\','')].fillna(0).astype(np.int32)
    return df[['email', file.replace('data\\','')]].drop_duplicates(subset="email")

def set_column_types(df):
    for col in df.columns:
        if col not in ['Area','codigo','Nome','email']:
            df[col]=df[col].fillna(0).astype(np.int32)

In [ ]:
for file in files:
    newdf = clean_file(file)
    newdf[file.replace('data\\','')]=newdf[file.replace('data\\','')].fillna(0).astype(np.int32)
    main_df = pd.merge(main_df,newdf, on="email", how="outer")

In [ ]:
main_df.columns

In [ ]:
# main_df['Negocio']=main_df['Negocio'].fillna('Outras').astype('category')
main_df['Area']=main_df['Area'].fillna('Outras').astype('category')
main_df['codigo']=main_df['codigo'].fillna(0).astype(np.int32)
main_df['Nome']=main_df['Nome'].fillna(0).astype('string')
main_df['email']=main_df['email'].fillna(0).astype('string')
set_column_types(main_df)

In [ ]:
main_df.info()

In [ ]:
column_list=list(main_df)
del column_list[0:4]

In [ ]:
main_df["Total"]=main_df[column_list].sum(axis=1)

In [ ]:
main_df.columns

In [ ]:
df_total = main_df[["Area","codigo","Nome","email","Total"]]
# df_total.to_csv('Email.csv')
# df_total.to_excel('Email.xlsx')

In [ ]:
sum = main_df.sum()

In [ ]:
sum = sum[1:-1]

In [ ]:
sum.plot(kind='bar',figsize=(20,10), rot=90, fontsize=14, color=color_azul_amazonico)

In [ ]:
df_nao_lidos = main_df[(main_df['Total']==0)].groupby(['Area']).count()['Total']

In [ ]:
df_lidos = main_df[(main_df['Total']>0)].groupby(['Area']).count()['Total']

In [ ]:
df_total = pd.merge(df_lidos, df_nao_lidos, how="outer", on="Area")
df_total.rename(columns={'Total_x': 'Leitores', 'Total_y': 'Nao_leitores'}, inplace=True)
df_total["Porcentagem_leitores"] = df_total["Leitores"] * 100 / (df_total["Leitores"]+df_total["Nao_leitores"])
df_total.round(2)

In [ ]:
df_total

In [ ]:
df_total.plot(kind='bar', figsize=(20, 10), y=["Leitores", "Nao_leitores"], stacked=True, rot=90, fontsize=14, color={"Leitores": color_azul_amazonico, "Nao_leitores": color_amarillo_solar})


In [ ]:
df_parcial = df_total[df_total["Porcentagem_leitores"]<75]
df_parcial

In [ ]:
#df_total.to_excel("result.xlsx", encoding="utf-8")


In [ ]:
df_parcial.plot(kind='bar',figsize=(20,10),y=["Leitores","Nao_leitores"], stacked=True, rot=90, fontsize=14, color={"Leitores":color_azul_amazonico,"Nao_leitores":color_amarillo_solar})

In [ ]:
df_total.columns

In [ ]:
eduardo_df = main_df[main_df["email"]=="nome@dominio.com"]